In [208]:
from numpy import *
from math import sqrt

# 1. 导入数据

In [209]:
def load_data(file_name):
    """
    导入数据
    input:  file_name(string):文件的存储位置
    output: samples(mat):特征
            labels(mat):标签
            n_class(int):类别的个数
    """
    # 1. 获取特征
    f = open(file_name)
    samples = []
    labels = []
    for line in f.readlines():
        samples_tmp = []
        lines = line.strip().split("\t")
        for i in range(len(lines) - 1):
            samples_tmp.append(float(lines[i]))  # 从字符串转为浮点型
        labels.append(int(lines[-1]))
        samples.append(samples_tmp)
    f.close()
    n_class = 1
    
    return mat(samples), mat(labels).transpose(), n_class

# 2. 训练网络模型
## 2.1 正向传播
### 2.1.1 计算隐含层的输出

In [210]:
def hidden_out(samples, center, rbf_delta):
    """
    rbf函数（隐含层神经元输出函数）
    input:samples(mat):数据特征
          center(mat):rbf函数中心
          rbf_delta(mat)：rbf函数扩展常数
    output：hidden_output（mat）隐含层输出
    """
    n_sample, n_feature = shape(samples)
    n_hidden, n_hidden_value = shape(center)  # (隐含层节点数, 隐含层节点对sample的每个feature的值的数量=n_feature)
    
    # 每个样本要和隐含层的每个节点做一次运算
    hidden_output = mat(zeros((n_sample, n_hidden)))  # 每一行的信息是一个样本的所有特征和每个隐含层节点的运算值
    for i in range(n_sample):
        for j in range(n_hidden):
            hidden_output[i, j] = exp(
                -1.0 * (samples[i,:] - center[j,:]) * (samples[i,:] - center[j,:]).T /
                (2 * rbf_delta[0,j] * rbf_delta[0,j])
            )  # 高斯核函数
            
    return hidden_output

### 2.1.2 计算输出层的输入

In [211]:
def predict_in(hidden_output, w):
    """
    计算输出层的输入
    input:  hidden_output(mat):隐含层的输出
            w(mat):隐含层到输出层之间的权重
    output: predict_in(mat):输出层的输入  (样本数, 输出节点数) 每一行的信息是样本的每个输出的预测值
    """
    predict_in = hidden_output * w
    return predict_in

### 2.1.3 集散输出层的输出

In [212]:
def linear(x):
    """
    输出层神经元激活函数
    input:  x(mat/float):自变量，可以是矩阵或者是任意实数
    output: 这里采用的是线性函数 y=x
    """
    return x

def predict_out(predict_in):
    """
    输出层的输出
    input:  predict_in(mat):输出层的输入
    output: result(mat):输出层的输出
    """
    result = linear(predict_in)
    return result

## 2.2 误差的反向传播

In [213]:
def bp(samples, label, predict, center, rbf_delta, w, alpha):
    """
    input:  label:真实值
            output_out:预测值
            center(mat):rbf函数中心
            rbf_delta(mat)：rbf函数扩展常数
            w(mat):隐含层到输出层之间的权重
            alpha:学习率
    """
    n_hidden, _ = shape(center)
    n_sample, _ = shape(samples)
    error = mat(label - predict)  # 每一行的信息是一个样本的每个预测输出的误差
    # 梯度下降之计算梯度
    for j in range(n_hidden):
        sum1 = 0.0
        sum2 = 0.0
        sum3 = 0.0
        for i in range(n_sample):
            sum1 += error[i,:] * exp(-1.0 * (samples[i,:]-center[j,:]) * (samples[i,:]-center[j,:]).T / (2 * rbf_delta[0,j] * rbf_delta[0,j])) * (samples[i,:]-center[j,:])
            sum2 += error[i,:] * exp(-1.0 * (samples[i,:]-center[j,:]) * (samples[i,:]-center[j,:]).T / (2 * rbf_delta[0,j] * rbf_delta[0,j])) * (samples[i,:]-center[j,:]) * (samples[i,:]-center[j,:]).T
            sum3 += error[i,:] * exp(-1.0 * (samples[i,:]-center[j,:]) * (samples[i,:]-center[j,:]).T / (2 * rbf_delta[0,j] * rbf_delta[0,j]))
        # 得梯度
        delta_center    = (w[j,:] / (rbf_delta[0,j] * rbf_delta[0,j])) * sum1
        delta_rbf_delta = (w[j,:] / (rbf_delta[0,j] * rbf_delta[0,j] * rbf_delta[0,j])) * sum2
        delta_w = sum3
        
        # 学习 根据梯度修正权重和rbf函数中心和扩展常数 
        center[j,:] = center[j,:] - alpha * delta_center  # ?
        rbf_delta[0,j] = rbf_delta[0,j] - alpha * delta_rbf_delta
        w[j,:] = w[j,:] - alpha * delta_w

## 2.1~2.2 训练

In [214]:
def bp_train(samples, labels, n_hidden, maxIteration, alpha, n_output):
    """
    计算隐含层的输入
    input:  samples(mat):数据特征
            labels(mat):标签
            n_hidden(int):隐含层的节点个数
            maxIteration(int):最大的迭代次数
            alpha(float):学习率
            n_output(int):输出层的节点个数
    output: center(mat):rbf函数中心
            rbf_delta(mat):rbf函数扩展常数
            w(mat):隐含层到输出层之间的权重
    """
    n_sample, n_feature = shape(samples)  # m为样本数, n为特征数
    
    ## 初始化参数 ##
    center = mat(random.rand(n_hidden, n_feature))  # (隐含层节点数, 样本特征数) rand的范围在0~1之间
    center = center * (8.0 * sqrt(6) / sqrt(n_feature + n_hidden)) - mat(ones((n_hidden, n_feature))) * (4.0 * sqrt(6) / sqrt(n_feature + n_hidden))  # 使初始化的参数落在一个更好的区间中, ?具体为什么8*sqrt(6)还没搞懂
    rbf_delta = mat(random.rand(1, n_hidden))  # (1, 隐含层节点数) 
    rbf_delta = rbf_delta * (8.0 * sqrt(6) / sqrt(n_feature + n_hidden)) - mat(ones((1, n_hidden))) * (4.0 * sqrt(6) / sqrt(n_feature + n_hidden))
    w = mat(random.rand(n_hidden, n_output))  # (隐含层节点数, 输出节点数)
    w = w * (8.0 * sqrt(6) / sqrt(n_hidden + n_output)) - mat(ones((n_hidden, n_output))) * (4.0 * sqrt(6) / sqrt(n_hidden + n_output))
    
    ## 训练 ##
    iter = 0
    while iter <= maxIteration:
        # 2.1 正向传播
        # 2.1.1 计算隐含层的输出
        hidden_output = hidden_out(samples, center, rbf_delta)  # 得到每个样本和每个隐含层节点运算的结果
        # 2.1.2 计算输出层的输入
        output_in = predict_in(hidden_output, w)
        # 2.1.3 集散输出层的输出, 激活
        predict = predict_out(output_in)
        
        # 2.2 反向传播
        bp(samples, label, predict, center, rbf_delta, w, alpha)  # 传的是引用(传可变对象进去, 就是传引用)
        
        if iter % 10 == 0:
            cost = get_cost(get_predict(samples, center, rbf_delta, w) - label)
            print ("\t-------- iter: ", iter, " ,cost: ",  cost)
        if cost < 3:   ###如果损失函数值小于3则停止迭
            break
        iter += 1
    
    return center, rbf_delta, w

## 其他函数

In [215]:
def get_cost(cost):
    """
    计算当前损失函数的值: 欧氏距离?
    input:  cost(mat):预测值与标签之间的差
    output: cost_sum / m (double):损失函数的值
    """
    n_sample, n_predict_output = shape(cost)
    
    cost_sum = 0.0
    for i in range(n_sample):
        for j in range(n_predict_output):
            cost_sum += cost[i,j] * cost[i,j]
    return cost_sum / 2

def get_predict(samples, center, rbf_delta, w):
    """
    计算最终的预测
    input:  samples(mat):特征
    output: 预测值
    """
    return predict_out(predict_in(hidden_out(samples, center, rbf_delta), w))

def err_rate(labels, predict):
    """
    计算训练样本上的错误率
    input:  labels(mat):训练样本的标签
            predict(mat):训练样本的预测值
    output: rate[0,0](float):错误率
    """
    n_sample, _ = shape(labels)
    for j in range(n_sample):
        if predict[j,0] > 0.5:
            predict[j,0] = 1.0
        else:
            predict[j,0] = 0.0

    err = 0.0
    for i in range(n_sample):
        if float(label[i, 0]) != float(predict[i, 0]):
            err += 1
    rate = err / m
    return rate

def save_model_result(center, rbf_delta, w, result):
    """
    保存最终的模型
    """
    def write_file(file_name, source):   
        f = open(file_name, "w")
        m, n = shape(source)
        for i in range(m):
            tmp = []
            for j in range(n):
                tmp.append(str(source[i, j]))
            f.write("\t".join(tmp) + "\n")
        f.close()
    
    write_file("center.txt", center)
    write_file("delta.txt", delta)
    write_file("weight.txt", w)
    write_file('train_result.txt',result)

# Main

In [216]:
# 1. 导入数据
print("--------- 1. load data ---------")
samples, labels, n_class = load_data("data.txt")
# 2. 训练网络模型
print ("--------- 2.training ------------")
center, rbf_delta, w = bp_train(samples, labels, n_hidden=20, maxIteration=5000, alpha=0.08, n_output=n_class)
# 3. 得到最终的预测结果
print ("--------- 3.get prediction ------------")
print ("训练准确性为：", (1 - err_rate(labels, get_predict(samples, center, rbf_delta, w))))
# 4. 保存最终的模型
print ("--------- 4.save model and result ------------")

--------- 1. load data ---------
--------- 2.training ------------
	-------- iter:  0  ,cost:  11409.183086026864
	-------- iter:  10  ,cost:  2.550120420553891e+46
	-------- iter:  20  ,cost:  1.2616060515459246e+87
	-------- iter:  30  ,cost:  4.0371293934046604e+125
	-------- iter:  40  ,cost:  8.587554180485625e+159
	-------- iter:  50  ,cost:  1.8503358529029686e+196


c:\program files\python37\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: overflow encountered in double_scalars


	-------- iter:  60  ,cost:  nan
	-------- iter:  70  ,cost:  nan
	-------- iter:  80  ,cost:  nan


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
samples, labels, n_class = load_data("data.txt")

In [ ]:
samples[0:199,0] 

In [ ]:
plt.plot(samples[0:200,0], samples[0:200,1], '+')
plt.plot(samples[200:400,0], samples[200:400,1], 'o')

In [217]:
# -*- coding: utf-8 -*-
"""
Created on Mon Dec 10 12:48:22 2018
@author: lj
"""
import tensorflow as tf
import numpy as np

def load_data(file_name):
    '''导入数据
    input:  file_name(string):文件的存储位置
    output: feature_data(mat):特征
            label_data(mat):标签
            n_class(int):类别的个数
    '''
    # 1、获取特征
    f = open(file_name)  # 打开文件
    feature_data = []
    label_tmp = []
    for line in f.readlines():
        feature_tmp = []
        lines = line.strip().split("\t")
        for i in range(len(lines) - 1):
            feature_tmp.append(float(lines[i]))
        label_tmp.append(int(lines[-1]))      
        feature_data.append(feature_tmp)
    f.close()  # 关闭文件
    
    # 2、获取标签
    m = len(label_tmp)
    n_class = len(set(label_tmp))  # 得到类别的个数
    
    label_data = np.mat(np.zeros((m, n_class)))
    for i in range(m):
        label_data[i, label_tmp[i]] = 1
    
    return np.mat(feature_data), label_data

class RBF_NN():
    def __init__(self,hidden_nodes,input_data_trainX,input_data_trainY):
        self.hidden_nodes = hidden_nodes #隐含层节点数
        self.input_data_trainX = input_data_trainX #训练样本的特征
        self.input_data_trainY = input_data_trainY #训练样本的标签
    
    def fit(self):
        '''模型训练
        '''
        # 1.声明输入输出的占位符
        n_input = (self.input_data_trainX).shape[1]
        n_output = (self.input_data_trainY).shape[1]
        X = tf.placeholder('float',[None,n_input],name = 'X')
        Y = tf.placeholder('float',[None,n_output],name = 'Y')
        # 2.参数设置
        ## RBF函数参数
        c = tf.Variable(tf.random_normal([self.hidden_nodes,n_input]),name = 'c')
        delta = tf.Variable(tf.random_normal([1,self.hidden_nodes]),name = 'delta')
        ## 隐含层到输出层权重和偏置
        W = tf.Variable(tf.random_normal([self.hidden_nodes,n_output]),name = 'W')
        b = tf.Variable(tf.random_normal([1,n_output]),name = 'b')
        # 3.构造前向传播计算图
        ## 隐含层输出
        ### 特征样本与RBF均值的距离
        dist = tf.reduce_sum(tf.square(tf.subtract(tf.tile(X,[self.hidden_nodes,1]),c)),1)  
        dist = tf.multiply(1.0,tf.transpose(dist))
        ### RBF方差的平方
        delta_2 = tf.square(delta)
        ### 隐含层输出
        RBF_OUT = tf.exp(tf.multiply(-1.0,tf.divide(dist,tf.multiply(2.0,delta_2))))           
        ## 输出层输入
        output_in = tf.matmul(RBF_OUT,W) + b
        ## 输出层输出
        y_pred = tf.nn.sigmoid(output_in)
        # 4.声明代价函数优化算法
        cost = tf.reduce_mean(tf.pow(Y - y_pred,2)) #损失函数为均方误差
        train_op = tf.train.GradientDescentOptimizer(0.05).minimize(cost) #优化算法为梯度下降法
        
        # 5.反向传播求参数
        trX = self.input_data_trainX
        trY = self.input_data_trainY
        
        with tf.Session() as sess:
            ##初始化所有参数
            tf.global_variables_initializer().run()
            for epoch in range(100):
                for i in range(len(trX)):
                    feed = {X:trX[i],Y:trY[i]}
                    sess.run(train_op,feed_dict = feed)
                if epoch % 10.0 == 0:
                    total_loss = 0.0
                    for j in range(len(trX)):
                        total_loss += sess.run(cost,feed_dict = {X:trX[j],Y:trY[j]})
                    print('Loss function at step %d is %s'%(epoch,total_loss / len(trX)))
            print('Training complete!')

            W = W.eval()
            b = b.eval()
            c = c.eval()
            delta = delta.eval()
            pred_trX = np.mat(np.zeros((len(trX),n_output)))
            ## 训练准确率
            correct_tr = 0.0
            for i in range(len(trX)):
                pred_tr = sess.run(y_pred,feed_dict = {X:trX[i]})
                pred_trX[i,:] = pred_tr
                if np.argmax(pred_tr,1) == np.argmax(trY[i],1):
                    correct_tr += 1.0
            print('Accuracy on train set is :%s'%(correct_tr/len(trX)))
            self.save_model('RBF_predict_results.txt',pred_trX)
            
    def save_model(self,file_name,weights):
        '''保存模型(保存权重weights)
        input：file_name(string):文件名
               weights(mat)：权重矩阵
        '''
        f_w = open(file_name,'w')
        m,n = np.shape(weights)
        for i in range(m):
            w_tmp = []
            for j in range(n):
                w_tmp.append(str(weights[i,j]))
            f_w.write('\t'.join(w_tmp)+'\n')
        f_w.close()
            
if __name__ == '__main__':
    print('------------------------1.Load Data---------------------')
    trainX, trainY = load_data('data.txt')
    print('------------------------2.parameter setting-------------')
    hidden_nodes = 20
    input_data_trainX = trainX
    input_data_trainY = trainY
    rbf = RBF_NN(hidden_nodes,input_data_trainX,input_data_trainY)
    rbf.fit()

W1028 02:23:16.551109 17556 deprecation.py:323] From c:\program files\python37\lib\site-packages\tensorflow\python\ops\math_grad.py:1205: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


------------------------1.Load Data---------------------
------------------------2.parameter setting-------------


InternalError: Blas GEMM launch failed : a.shape=(1, 20), b.shape=(20, 2), m=1, n=2, k=20
	 [[node MatMul (defined at <ipython-input-217-0f2f024f8d3a>:70) ]]

Errors may have originated from an input operation.
Input Source operations connected to node MatMul:
 Exp (defined at <ipython-input-217-0f2f024f8d3a>:68)	
 W/read (defined at <ipython-input-217-0f2f024f8d3a>:58)

Original stack trace for 'MatMul':
  File "c:\program files\python37\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\program files\python37\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\program files\python37\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\program files\python37\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "c:\program files\python37\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  Fil